Trying to use an SVR for localizations instead of chi squared


In [2]:
from numpy import rad2deg,deg2rad,pi,sqrt,add,array,average
from healpy import ang2vec, newvisufunc
import numpy as np
from healpy import nside2npix,pix2ang
import burstutils as bf

In [18]:
def relative_scale(detarray):
    """Takes the absolute # of counts in each detector, and converts to a relative scale. 
    """
    
    newverz = []
    
    for val in detarray:
        if min(detarray) == max(detarray):
            z = 0
        else: 
            z = (val - min(detarray))/(max(detarray)-min(detarray))

        newverz.append(z)
    return newverz

In [19]:
#Needs to know how to point to healpy 
from healpy import nside2npix,pix2ang
class Sky():
    """
    Generates an array of GRB's given 
    certains strength at different sky positions.
    
    Output should be an array. 
    """
    def __init__(self,NSIDE,strength):

        #depending on NSIDE, there will be anywhere from 12 to infinite spots on the sky w/ GRBs
        self.Ao = strength
        self.pixels = nside2npix(NSIDE)

        #want to convert these pixels into theta phi coords. 
        self.sourceangs = []
        for i in range(self.pixels):
            self.sourceangs.append(pix2ang(NSIDE,i))


In [43]:
GRBs = Sky(NSIDE=32,strength=500)

In [44]:
len(rad2deg(GRBs.sourceangs))

12288

In [45]:


#sometimes one way works, sometimes another one does. Here's a workaround. 
try:
    from NoahSim import burstutils as bf
except:
    import burstutils as bf

from random import gauss
import statistics as s
import matplotlib.pyplot as plt

#making classes of objects, allows for different instances of burstcube, easy to compare. 
class FastCube():

    def __init__(self,background,dettilt,alternating=False):
        if alternating == False:
            self.tilt = deg2rad(dettilt)
            self.tiltA = self.tiltB = self.tiltC = self.tiltD = self.tilt
        
        else:
            self.tiltB = (float(input("Please enter the second tilt (deg) ")))
            self.tiltB = deg2rad(self.tiltB)
            self.tiltC = self.tiltA = deg2rad(dettilt)
            self.tiltD = self.tiltB
        
        self.zenith = [0 , 0]
        self.bg = background


        
    @property
    def detA(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltA , self.zenith[1] ]
    @property 
    def detB(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltB , self.zenith[1] + pi/2 ]
    @property
    def detC(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltC , self.zenith[1] + pi ]
    @property 
    def detD(self):
        """BurstCube is composed of 4 separate scintillators to detect and localize events. 
        In this software package, they are labelled A through D. 
        """
        return [ self.zenith[0] + self.tiltD , self.zenith[1] + 3*pi/2 ]
    @property
    def normA(self):
        return  ang2vec(self.detA[0],self.detA[1])
    @property 
    def normB(self):
        return  ang2vec(self.detB[0],self.detB[1])
    @property
    def normC(self):
        return  ang2vec(self.detC[0],self.detC[1])
    @property 
    def normD(self):
        return  ang2vec(self.detD[0],self.detD[1])

    
    @property
    def dets(self):
        return [self.normA,self.normB,self.normC,self.normD] 

    #now that the properties of burstucbe have been designed, now its time to test the model's localization capabilities    

    def response2GRB(self, GRB,test=True,talk=False):   #is this how I inherit? 

    #first need to include the GRB.
       
        """
        Using x, respond2GRB will determine the sky position of an array of GRB sources assuming some inherent background noise within 
        detectors, along with fluctuations of either Gaussian or Poissonian nature. 

        Parameters
        ----------
        GRB : object
            An instance of the separately defined "GRBs" class that contains a number of evenly spaced sky positions of a given strength. 
        
        test : boolean 
            For sanity purposes, if the simulation seems to give unrealistic results, switching to test mode allows for much quicker sampling, allowing it easier to spot potential errors. 
        
        
        talk : boolean
            If desired, prints position by position results. 
        
        Returns
        ----------
        localizationerrors : array
            numpy array that contains the average localization uncertainty at each sky position. 
        
        Additionally, response2GRB will print the sky position it is currently sampling, along with the average offset of localizations at that spot. 
        
        """
        
        if test:
            sample = 1
            bottheta = 0
            toptheta = 180
            botphi = 0 
            topphi = 360
            botA = 10
            topA = 1000
            ntheta = 20   #over sky chi points
            nphi = 37
            nA = 100

        else:
            sample = len(GRB.sourceangs)   #number of GRBs you're testing
            bottheta = 0   #zenith
            toptheta = 120  #(elevation) range of theta values   horizon
            ntheta = 31   #over sky chi points  #binning

            botphi = 0 #azimuthal angles
            topphi = 360
            botA = 200  #range of amplitudes/strength of source it tries to match
            topA = 1000   #counts above background 
            nphi = 120
            nA = 12 
        self.localizationerrors = []   
        X = []
        for i in range(len(GRB.sourceangs)):
            
            sourceAng = GRB.sourceangs[i]
            if talk:
                print("Testing " + str(rad2deg(sourceAng)))
           #this check passes.       

            
           # print("Testing at " + str(np.rad2deg(GRB.sourceangs)))
            sourcexyz = ang2vec(sourceAng[0],sourceAng[1]) #cartesian position of the burst
                    #here's where I define how many times that is
                
            sepA=bf.angle(sourcexyz,self.normA)
            xA = bf.look_up_A(self.normA,sourcexyz)
                   # print("separation from A is " + str(np.rad2deg(sepA)))
                   #this check passes.  
               
            dtheoryA=GRB.Ao*bf.response(sepA,xA)  #still need to define strength, brb and gonna do that 
            if type(dtheoryA) == np.float64:
                dtheoryA = np.array([dtheoryA])
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 
                    

            sepB=bf.angle(sourcexyz,self.normB)
            xB = bf.look_up_B(self.normB,sourcexyz)

                   # print("separation from B is " + str(np.rad2deg(sepB)))
                   #this check passes.  
               
            dtheoryB=GRB.Ao*bf.response(sepB,xB)  
                    #still need to define strength, brb and gonna do that 
            if type(dtheoryB) == np.float64:
                dtheoryB = np.array([dtheoryB])

                     
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too. 

                


            sepC=bf.angle(sourcexyz,self.normC)
                   # print("separation from C is " + str(np.rad2deg(sepC)))
                   #this check passes.  
            xC =  bf.look_up_C(self.normC,sourcexyz)
            dtheoryC=GRB.Ao*bf.response(sepC,xC)  #still need to define strength, brb and gonna do that 
            if type(dtheoryC) == np.float64:
                dtheoryC = np.array([dtheoryC])
                   # print("dtheory test: " + str(dtheory))
                    # this check passes too
                
                

                
            sepD=bf.angle(sourcexyz,self.normD)
                   # print("separation from D is " + str(np.rad2deg(sepD)))
                   #this check passes.  
            xD = bf.look_up_D(self.normD,sourcexyz)
            dtheoryD=GRB.Ao*bf.response(sepD,xD)  #still need to define strength, brb and gonna do that 
            if type(dtheoryD) == np.float64:
                dtheoryD = np.array([dtheoryD])
                     
                   # print("dtheory test: " + str(dtheory))
            X1 = np.concatenate((dtheoryA,dtheoryB))
            X2 = np.concatenate((dtheoryC,dtheoryD))
            X3 = np.concatenate((X1,X2))
            X3 = relative_scale(X3)
            X.append(X3)   
        return np.array(X)
    

In [46]:
cubesat = FastCube(1000,45)

In [47]:
Xs = cubesat.response2GRB(GRBs)


In [51]:
ys = rad2deg(GRBs.sourceangs)

In [52]:
Xs

array([[1., 1., 0., 0.],
       [0., 1., 1., 0.],
       [0., 0., 1., 1.],
       ...,
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [53]:
ys

array([[  1.46197116,  45.        ],
       [  1.46197116, 135.        ],
       [  1.46197116, 225.        ],
       ...,
       [178.53802884, 135.        ],
       [178.53802884, 225.        ],
       [178.53802884, 315.        ]])

Example SVR
-----------

from sklearn import svm
 X = [[0, 0], [2, 2]]
 
 y = [0.5, 2.5]
 
 clf = svm.SVR()

clf.fit(X, y) 

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)
 clf.predict([[1, 1]])
array([ 1.5])

In [52]:
X = [[0, 0], [2, 2],[2,2]]

In [53]:
np.shape(X)

(3, 2)

In [54]:
from sklearn import multioutput
from sklearn import svm

In [55]:
model = svm.SVR()

In [56]:
clf = multioutput.MultiOutputRegressor(estimator=model)

In [57]:
from sklearn import preprocessing, cross_validation

X_train,X_test,y_train,y_test = cross_validation.train_test_split(Xs,ys,test_size=0.27)


In [58]:
X_test[21]

array([1.        , 1.        , 0.75248373, 0.        ])

In [60]:
clf.fit(X_train,y_train)

MultiOutputRegressor(estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False),
           n_jobs=1)

In [61]:
clf.score(X_test,y_test)

0.34020206678234755

In [80]:
test  = Xs[4]
test = test.reshape(1,-1)


In [81]:
clf.predict(test)

array([[ 75.46632338, 196.38857207]])

In [83]:
ys[4]

array([ 2.92418036, 22.5       ])

In [65]:
            
                #coarse to fine optimization
                chiA = bf.quad_solver(detcountsA,self.normA,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,A=True)
                chiB = bf.quad_solver(detcountsB,self.normB,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,B=True)
                chiC = bf.quad_solver(detcountsC,self.normC,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,C=True)
                chiD = bf.quad_solver(detcountsD,self.normD,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA,self.bg,D=True)
                
                chisquared = add(add(chiA,chiB),add(chiC,chiD)) #adds it all up for total chi2
                
                #print("Chi squareds: " +str(chisquared))
                
                
                thetaloc, philoc, Aguess = bf.indexer(chisquared,bottheta,toptheta,botphi,topphi,botA,topA,ntheta,nphi,nA)
                recvec = ang2vec(deg2rad(thetaloc),deg2rad(philoc))
                locoffset = rad2deg(bf.angle(sourcexyz,recvec))
               # print("Loc offset = " + str(locoffset) + " deg")
                
                locunc.append(locoffset)
                loop +=1
            if talk:
                print("Avg loc offset = " + str(average(locunc)) + " deg.")

            self.localizationerrors.append(s.mean(locunc))
        return self.localizationerrors

IndentationError: unexpected indent (<ipython-input-65-29a2192b3c3b>, line 3)